# 

# Table of Contents

<a id="toc"></a>
- [1. Competition 설명 및 개발 목표](#1)
    
- [2. 데이터 구조 설명 ](#2)

    - [2.1 한국어 예시 데이터 ](#2.1)
    
    - [2.2 실제 데이터 ](#2.2)
    
    - [2.3 CPC scheme ( context ) ](#2.3)
    
    - [2.4 Score 의미](#2.4)

- [3. 평가 방법](#3)

- [4. 추가 설명](#4)
    

<a id="1"></a>

## 1. Competition 설명 및 개발 목표 

- 미국 특허청 (USPTO)은 많은 양의 특허 문서를 보유하고 있음.

- 보유한 특허 문서에서 **주요 키워드를 검색** 했을 때, **유사 특허를 검색하기 위해 사용할 것** 으로 활용할 것으로 보임.

- 이번 Competition의 목표는 입력된 **"주요 키워드"와 기존 문서에 등장하는 "특정 키워드" 간의 유사도를 판단**하는 것임.

- 즉, **검색하고자 하는 단어 ( anchor )와 비교하고자 하는 단어 ( target )의 유사도를 판단** 해야함.



<a id="2"></a>

### 2. 데이터 구조 설명

<a id="2.1"></a>

#### 2.1 한국어 예시 데이터

- id : 해당 데이터의 고유 번호

- anchor : 입력된 키워드

- target : 비교하고자 하는 키워드

- context : 쉬운 이해를 위해 context의 예시로 설명

    - Z : 재료
    
    - Z19 : 금속
    
    - Z20 : 원단
    
    - ...

- score : 유사도 

|  id   |      anchor      |  target | context | score |
|:----------:|:--------------------:|:----------:|:----:| :----:|
| 37d61fd2272659b2 |  강한 성질의 재료 | 강한 성질의 재료들 | Z19 ( 금속 ) | 1.0 |
| 37d61fd2272659b3 |  강한 성질의 재료 | 강성을 갖는 재료 | Z19 ( 금속 ) | 0.75 |
| 37d61fd2272659b1 |  강한 성질의 재료 | 강철 | Z19 ( 금속 ) | 0.5 |
| 37d61fd2272659b4 |  강한 성질의 재료 | 약한 성질의 재료 | Z19 ( 금속 ) | 0.25 |
| 37d61fd2272659b5 |  강한 성질의 재료 | 나일론 | Z19 ( 금속 ) | 0.25 |
| 37d61fd2272659b6 |  강한 성질의 재료 | 사무용 가구 | Z19 ( 금속 ) | 0.00 |
| 37d61fd2272659b8 |  강한 성질의 재료 | 강한 성질의 재료들 | Z20 ( 원단 ) | 1.0 |
| 37d61fd2272659b9 |  강한 성질의 재료 | 강성을 갖는 재료 | Z20 ( 원단 ) | 0.75 |
| 37d61fd2272659b7 |  강한 성질의 재료 | 나일론 | Z20 ( 원단 ) | 0.5 |
| 37d61fd2272659ba |  강한 성질의 재료 | 약한 성질의 재료 | Z20 ( 원단 ) | 0.25 |
| 37d61fd2272659bb |  강한 성질의 재료 | 강철 | Z20 ( 원단 ) | 0.25 |
| 37d61fd2272659bc |  강한 성질의 재료 | 사무용 가구 | Z20 ( 원단 ) | 0.00 |
| ... |  ... | ... | ... | ... |


<a id="2.2"></a>
#### 2.2 실제 데이터

In [ ]:
import os
import pandas as pd

INPUT_DIR = '../input/us-patent-phrase-to-phrase-matching/'

train = pd.read_csv(INPUT_DIR+'train.csv')

display(train.head())

<a id="2.3"></a>

#### 2.3 Context ( CPC Scheme )란 무엇인가?

- 미국 특허청에서 각 특허의 성격에 따라 분류한 표

- CPC Scheme에는 ***'A01B 15/12'*** 와 같이 아주 세세한 정보까지 있지만 이번 Competition에서는 맨 앞 세자리만 사용

- 앞자리 부터 ***대분류(A) > 중분류(01) > 소분류(B) > 상세 분류(15/12)*** 를 의미함.

- 즉, ***대분류와 중분류*** 까지만 사용.

- 자세한 내용은 https://www.uspto.gov/web/patents/classification/cpc/html/cpc.html 에서 확인 가능함.

- ***CPC 대분류***

|  대분류   |      내용      |  번역 | 
|:----------:|:--------------------:|:--------------------:|
|A |HUMAN NECESSITIES|인간의 필수품|
|B |PERFORMING OPERATIONS; TRANSPORTING| 작업 수행; 운송 |
|C |CHEMISTRY; METALLURGY| 화학; 야금 |
|D |TEXTILES; PAPER| 섬유; 종이 |
|E |FIXED CONSTRUCTIONS| 고정된 구조물 |
|F |MECHANICAL ENGINEERING; LIGHTING; HEATING; WEAPONS; BLASTING| 기계 공학; 조명; 난방; 무기; 폭파 |
|G |PHYSICS| 물리학 |
|H |ELECTRICITY| 전기 |
|Y |GENERAL TAGGING OF NEW TECHNOLOGICAL ...| 신기술 등 ... |

- ***CPC A에 대한 중분류***

|  대분류   |      내용      | 번역 |  
|:----------:|:--------------------:|:--------------------:|
| A01 | AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING |농업; 임학; 축산; 수렵; 트래핑; 어업|
| A21 | BAKING; EDIBLE DOUGHS | 빵 굽기; 식용 반죽 | 
| A22 | BUTCHERING; MEAT TREATMENT; PROCESSING POULTRY OR FISH | 도살; 육류 처리; 가금류 또는 생선 가공 |
| A23 | FOODS OR FOODSTUFFS; TREATMENT THEREOF, NOT COVERED BY OTHER CLASSES | 음식 또는 식품; 앞에 다루어진 것들, 이 밖의 다른 것들 |
| ... | ... | ... |

<a id="2.4"></a>

#### 2.4 Score의 의미

- 1.0 : 매우 밀접하게 유사함. ( 복수형, and, or 과 같은 불용어를 제외했을 때 매우 일치함. )

- 0.75 : 동의어에 가깝게 유사함. ( 동의어, 약어, 풀이된 약어 등 )

- 0.5 : 동의어는 아니지만 해당 분류 내에서 유사한 속성을 갖게 유사함.

- 0.25 : 반의어와 상위 수준의 CPC 상위 분류에서는 존재하지만 세부 분류에서는 존재하지 않음. 

- 0.0 : 아예 관련이 없는 단어

##### Score와 함께 보는 Data 예시

- Z : 재료

|  id   |      anchor      |  target | context | score |
|:----------:|:--------------------:|:----------:|:----:| :----:|
| 37d61fd2272659b2 |  강한 성질의 재료 | 강한 성질의 재료들 | Z19 ( 금속 ) | 1.0 |
| 37d61fd2272659b3 |  강한 성질의 재료 | 강성을 갖는 재료 | Z19 ( 금속 ) | 0.75 |
| 37d61fd2272659b1 |  강한 성질의 재료 | 강철 | Z19 ( 금속 ) | 0.5 |
| 37d61fd2272659b4 |  강한 성질의 재료 | ***약한 성질의 재료*** | Z19 ( ***금속*** ) | 0.25 |
| 37d61fd2272659b5 |  강한 성질의 재료 | ***나일론*** | Z19 ( ***금속*** ) | 0.25 |
| 37d61fd2272659b6 |  강한 성질의 재료 | 사무용 가구 | Z19 ( 금속 ) | 0.00 |
| 37d61fd2272659b8 |  강한 성질의 재료 | 강한 성질의 재료들 | Z20 ( 원단 ) | 1.0 |
| 37d61fd2272659b9 |  강한 성질의 재료 | 강성을 갖는 재료 | Z20 ( 원단 ) | 0.75 |
| 37d61fd2272659b7 |  강한 성질의 재료 | 나일론 | Z20 ( 원단 ) | 0.5 |
| 37d61fd2272659ba |  강한 성질의 재료 | ***약한 성질의 재료*** | Z20 ( ***원단*** ) | 0.25 |
| 37d61fd2272659bb |  강한 성질의 재료 | ***강철*** | Z20 ( ***원단*** ) | 0.25 |
| 37d61fd2272659bc |  강한 성질의 재료 | 사무용 가구 | Z20 ( 원단 ) | 0.00 |
| ... |  ... | ... | ... | ... |


#### ※ 따라서 Context ( CPC Scheme )에 대한 각각의 대분류, 중분류의 정보도 매우 중요함.

<a id="3"></a>

### 3. 평가 방법

- Pearson correlation coefficient를 사용 :


$$
    P C C(X, Y)=\frac{C O V(X, Y)}{S D_{x} * S D_{y}} 
$$

$$
    P C C (X, Y)= \frac{\sum_{i=1}^{n}\left(\left(x_{i}-m_{x}\right) *\left(y_{i}-m_{y}\right)\right)}{\sqrt{  \sum_{i=1}^{n}\left(x_{i}-{m_{x}}\right)^{2} \sum_{i=1}^{n}\left(y_{i}-m_{y}\right)^{2} }}
$$


- 1차 함수로 표현 가능한 분포 x, y에 대한 상관 계수

- ***입력된 두개의 변수에 대해 오차가 얼마나 적으면서 상관 관계가 있는가를 볼 수 있는 방법***

- PCC(X,Y)가 1에 가깝다면 : 두 X,Y는 아주 강력한 양의 상관 관계가 있다. ( 모든 데이터가 같다. )

- PCC(X,Y)가 0에 가깝다면 : 두 X,Y는 관계가 없다.

- PCC(X,Y)가 -1에 가깝다면 : 두 X,Y는 아주 강력한 음의 상관 간계가 있다. ( 모든 데이터가 반대이다. )

In [ ]:
import numpy as np

def pearson(x, y):
    mx = np.mean(x)
    my = np.mean(y)
    
    std_x = 0.0
    std_y = 0.0
    cov = 0.0
    eps = 1e-10
    
    for _x in x :
        std_x = std_x + (_x - mx )*(_x - mx) 
        
    for _y in y :
        std_y = std_y + (_y - my)*(_y - my)
    
    div = np.sqrt(std_x * std_y) + eps
    
    for idx in range(len(x)) :
        cov = cov + (x[idx]-mx)*(y[idx]-my) 
    
    r = cov/div
    
    return r



In [ ]:

w = np.array([0.5, 0.5, 0.5, 0.5, 0.5]) # uniform 예시

x = np.array([0.1, 0.4, 0.4, 0.5, 0.9]) # 모델의 응답값 예시

y = np.array([0.0, 0.25, 0.5, 0.75, 1.0]) # 정답지

z = np.array([1.0, 0.75, 0.5, 0.25, 0.0]) # 거꾸로된 정답지

print('pearson(w,y) : ', f'{pearson(w,y):.3f}')
print()
print('pearson(x,y) : ', f'{pearson(x,y):.3f}')
print()
print('pearson(x,x) : ', f'{pearson(x,x):.3f}')
print()
print('pearson(y,z) : ', f'{pearson(y,z):.3f}')

<a id="4"></a>

### 4. 추가 설명

#### 4.1 score 별 데이터 분포

In [ ]:
#x = train['score'].hist()
#display(train['score'].value_counts())
import matplotlib.pyplot as plt
keys = [0.00, 0.25, 0.5, 0.75, 1.0]
cnt = train['score'].value_counts()
counters = []
for key in keys :
    counters.append(cnt[key])

plt.bar(range(len(keys)), counters)
plt.xticks(range(len(keys)), keys)
plt.ylabel('freq')
plt.xlabel('score')
plt.show()

#### 4.2 대분류 별 데이터 분포

In [ ]:

keys = ['A','B','C','D','E','F','G','H' ]
cnt = train['context'].apply(lambda x: x[0]).value_counts()
counters = []
data = {}
for key in sorted(keys) :
    counters.append(cnt[key])
    #data[key] = cnt[key]
    #print(key, ' : ', cnt[key])
plt.bar(range(len(keys)), counters)
plt.xticks(range(len(keys)), keys)
plt.ylabel('freq')
plt.xlabel('context(Section)')
plt.show()


#### 4.3 추가로 사용할 수 있을 법한 데이터

##### ⓐ Big patent Dataset ( https://www.tensorflow.org/datasets/catalog/big_patent )

- 약 10GB의 특허 관련 데이터 뭉치

- CPC code에 따른 abstract, description을 제공함.

</br>

##### ⓑ huggingface에서 제공하는 patent 관련 LM

- https://huggingface.co/google/bigbird-pegasus-large-bigpatent

- https://huggingface.co/anferico/bert-for-patents

- https://huggingface.co/AI-Growth-Lab/PatentSBERTa

- 아쉽게도 갓 deberta 기반의 공개된 patent 관련 LM은 없음. ( deberta 기반으로 추가 학습을 진행 )

#### 4.4 Jaccard Similarity로 본 Data EDA

$$
    Jaccard Similarity(X, Y)=\frac{len(X ∩ Y)}{len(X ∪ B)} 
$$

- 4.4.1 Anchor, Target 의 and, or 키워드 제거

- 4.4.2 Anchor와 Target을 split(' ')

- 4.4.3 Jaccard Similiarity 계산

- 4.4.4 display( Jaccard Simliarity == 1.0 and train['score'] != 1.0 )



In [ ]:
for index, row in train.iterrows():    
    anchor = set(row['anchor'].replace(' and ','').strip().replace(' or ','').strip().split())
    target = set(row['target'].replace(' and ','').strip().replace(' or ','').strip().split())    
    score = row['score']
    jaccard = len(anchor & target) / len( anchor | target)
    if score < 0.95 and jaccard > 0.95: 
        print(f'score({score:<5})\tjaccard : ({jaccard:^5}) \t', f"anchor : {row['anchor']:<30} target : {row['target']:<30} context : {row['context']:<30}"  )        

    
    
    


### 5. 공개된 모델의 input 과 output

#### 5.1 공개된 모델의 Input

    - Input type : "<s>anchor<sep>target<sep><context text></s>"
    
    - example data > anchor : "abatement", target : "abatement of pollution", context :A23 score : 0.50
    
    - input for LM > <s>abatement<sep>abatement of pollution<sep>HUMAN NECESSITIES; FOODS OR FOODSTUFFS; TREATMENT THEREOF, NOT COVERED BY OTHER CLASSES </s>
    
#### 5.2 공개된 모델의 Output :

    - Classification : 
    
        - Linear( hidden_dim, 5 ) # 0 : 0.0, 1 : 0.25, 2 : 0.5, 3 : 0.75, 4 : 1.0

        - MSE(label, pred)

    - Regression : 
    
        - Linear( hidden_dim, 1 ) : score

        - MSE(0.5, pred)
    